<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# Lab 2 - Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [8]:
%sql sqlite:///movies.db

'Connected: @movies.db'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [4]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.db
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [6]:
%%sql
SELECT *
FROM movies

 * sqlite:///movies.db
Done.


imdb_key,movie_name,production_year,running_time
tt0111161,The Shawshank Redemption,1994,144
tt0816692,Interstellar,2014,169
tt0892769,How to Train your Dragon,2010,98
tt2084970,The Imitation Game,2014,114
tt1201607,Harry Potter and the Deathly Hallows: Part 2,2011,130
tt0167260,Lord of the Rings: The Return of the King,2003,201


+ Show the performance dates for one of the movies.

In [11]:
%%sql
SELECT  start_time, start_date
FROM    performances
WHERE   imdb_key = "tt0111161"

 * sqlite:///movies.db
Done.


start_time,start_date
19:30,2019-02-16
20:30,2019-02-16
19:30,2019-02-14


+ Show all data concerning performances at a given theatere
  on a given date.

In [16]:
%%sql
SELECT  *
FROM    theaters
JOIN    performances
USING   (theater_name)
WHERE   start_date = "2019-02-16" AND theater_name = "Filmstaden Lund"

 * sqlite:///movies.db
Done.


theater_name,capacity,performance_id,start_time,start_date,imdb_key
Filmstaden Lund,237,19f48fe1d4f6d5476532ad8a0ea4093b,19:30,2019-02-16,tt0111161
Filmstaden Lund,237,3b579db243c51d501e18f190dcf13a70,20:30,2019-02-16,tt0111161


+ List all customers

In [ ]:
%%sql


+ List all tickets

In [ ]:
%%sql


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [ ]:
%%sql


In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2019), we can't do that, instead
  we can the following idea: each row in a SQLite3 table has
  a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries, unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [ ]:
%%sql


+ Try to insert two movie theaters with the same name (this
  should fail).

In [ ]:
%%sql


+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [ ]:
%%sql


+ Create a ticket where either the user or the performance
  doesnâ€™t exist.

In [ ]:
%%sql
